<a href="https://colab.research.google.com/github/jittinabraham/leaflet-product-classification/blob/main/leafletproductsimagemodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
   !pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=1be1f2f3b7b31e9e1f0d2c011784a7e7a61b1713d2b90e380e00a47c51f0f7cd
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [2]:
!wget https://zenodo.org/record/7869954/files/products_leaflets_256.zip

--2023-11-19 20:12:15--  https://zenodo.org/record/7869954/files/products_leaflets_256.zip
Resolving zenodo.org (zenodo.org)... 188.184.103.159, 188.184.98.238, 188.185.79.172, ...
Connecting to zenodo.org (zenodo.org)|188.184.103.159|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/7869954/files/products_leaflets_256.zip [following]
--2023-11-19 20:12:16--  https://zenodo.org/records/7869954/files/products_leaflets_256.zip
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 1171218652 (1.1G) [application/octet-stream]
Saving to: ‘products_leaflets_256.zip’

products_leaflets_2 100%[===================>]   1.09G  19.4MB/s    in 59s     

2023-11-19 20:13:16 (18.8 MB/s) - ‘products_leaflets_256.zip’ saved [1171218652/1171218652]



In [3]:
!unzip products_leaflets_256.zip


Streaming output truncated to the last 5000 lines.
  inflating: products_leaflets_256/train/738/39037.jpg  
  inflating: products_leaflets_256/train/738/39038.jpg  
  inflating: products_leaflets_256/train/738/39040.jpg  
  inflating: products_leaflets_256/train/738/39041.jpg  
  inflating: products_leaflets_256/train/738/39042.jpg  
  inflating: products_leaflets_256/train/738/39043.jpg  
  inflating: products_leaflets_256/train/738/39050.jpg  
  inflating: products_leaflets_256/train/738/39051.jpg  
  inflating: products_leaflets_256/train/738/39052.jpg  
  inflating: products_leaflets_256/train/738/39053.jpg  
  inflating: products_leaflets_256/train/738/39054.jpg  
  inflating: products_leaflets_256/train/738/39055.jpg  
  inflating: products_leaflets_256/train/738/39056.jpg  
  inflating: products_leaflets_256/train/738/39057.jpg  
  inflating: products_leaflets_256/train/738/39058.jpg  
  inflating: products_leaflets_256/train/738/39059.jpg  
  inflating: products_leaflets_256/tr

In [4]:
from tensorflow.keras.layers import Input
import tensorflow as tf
from keras.models import Sequential
import keras
import pandas as pd
import os
import random
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('ggplot')
#import cv2



from keras.optimizers import Adam



%matplotlib inline


In [5]:
import os
import csv

# Path to the root directory containing the class folders (0-831)
root_directory = '/content/products_leaflets_256/train'

# Output CSV file name
csv_filename = 'image_class_mapping.csv'

# Open the CSV file for writing
with open(csv_filename, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)

    # Write the header row
    csv_writer.writerow(['Image_Name', 'Class_Name'])

    # Loop through each class folder
    for class_folder in os.listdir(root_directory):
        class_path = os.path.join(root_directory, class_folder)

        # Check if it's a directory
        if os.path.isdir(class_path):
            # Loop through each image in the class folder
            for image_name in os.listdir(class_path):
                # Write the image name and class name to the CSV file
                csv_writer.writerow([image_name, class_folder])


In [6]:
import os
import shutil
import random

# Set the paths
test_data_path = '/content/products_leaflets_256/test'
validation_data_path = '/content/products_leaflets_256/validation'

# Create the validation directory if it doesn't exist
os.makedirs(validation_data_path, exist_ok=True)

# Iterate through subfolders in the test dataset
for subfolder in os.listdir(test_data_path):
    subfolder_path = os.path.join(test_data_path, subfolder)

    # Check if it's a directory
    if os.path.isdir(subfolder_path):
        # Get a list of images in the subfolder
        images = [f for f in os.listdir(subfolder_path) if f.endswith(('.jpg', '.jpeg', '.png'))]

        # Calculate the number of images to move (50%)
        num_images_to_move = int(0.5 * len(images))

        # Randomly select images to move
        images_to_move = random.sample(images, num_images_to_move)

        # Create the corresponding subfolder in the validation directory
        validation_subfolder_path = os.path.join(validation_data_path, subfolder)
        os.makedirs(validation_subfolder_path, exist_ok=True)

        # Move selected images to the validation subfolder
        for image_to_move in images_to_move:
            source_path = os.path.join(subfolder_path, image_to_move)
            destination_path = os.path.join(validation_subfolder_path, image_to_move)
            shutil.move(source_path, destination_path)

print("Validation dataset created successfully.")


Validation dataset created successfully.


In [7]:
import os
import shutil

# Path to the 'train' folder
train_folder = '/content/products_leaflets_256/train'

# Path to the new folder
new_folder = '/content/products_leaflets_256/temp'

# Create the new folder if it doesn't exist
if not os.path.exists(new_folder):
    os.makedirs(new_folder)

# Loop through each subfolder in the 'train' folder
for subfolder in os.listdir(train_folder):
    subfolder_path = os.path.join(train_folder, subfolder)

    # Check if it's a directory
    if os.path.isdir(subfolder_path):
        # Move all files from the subfolder to the new folder
        for file_name in os.listdir(subfolder_path):
            file_path = os.path.join(subfolder_path, file_name)
            shutil.move(file_path, new_folder)

# Optionally, remove the empty subfolders (if needed)
for subfolder in os.listdir(train_folder):
    subfolder_path = os.path.join(train_folder, subfolder)
    if os.path.isdir(subfolder_path):
        os.rmdir(subfolder_path)


In [8]:
import pandas as pd

# Load the CSV file into a DataFrame
csv_file_path = '/content/image_class_mapping.csv'
df = pd.read_csv(csv_file_path)

# Shuffle the rows
shuffled_df = df.sample(frac=1).reset_index(drop=True)

# Save the shuffled DataFrame back to a CSV file
shuffled_csv_file_path = '/content/train_shuffle.csv'
shuffled_df.to_csv(shuffled_csv_file_path, index=False)


In [9]:
import os
from PIL import Image
import numpy as np

# Set the paths to your data folders
data_folder = '/content/products_leaflets_256/validation'
batch1_folders = range(0, 832)  # Folders from 0 to 400


# Function to load images from a list of folders
def load_images_from_folders(folders):
    x_train = []
    y_label = []

    for folder in folders:
        folder_path = os.path.join(data_folder, str(folder))
        for filename in os.listdir(folder_path):
            image_path = os.path.join(folder_path, filename)
            img = Image.open(image_path)
            img = img.resize((desired_width, desired_height))  # Resize if needed
            x_train.append(np.array(img))
            y_label.append(folder)

    return np.array(x_train), np.array(y_label)

In [10]:

desired_width = 256
desired_height = 256
x_validation, y_validation = load_images_from_folders(batch1_folders)

In [11]:
from tensorflow.keras.utils import to_categorical
# Number of classes (378 in your case)
num_classes = 832

# Subtract one from the class labels to account for the shift


# One-hot encode the shifted labels

y_label_encoded_validation = to_categorical(y_validation, num_classes)

In [64]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, AveragePooling2D,Flatten

In [13]:
# Assuming you have a CSV file for training data with columns 'Image_Name' and 'Class_Label'
train_csv_file = '/content/image_class_mapping.csv'

# Assuming you have a directory with training images
train_images_dir = '/content/products_leaflets_256/temp'

In [14]:
# Load training data from CSV
train_df = pd.read_csv(train_csv_file)

# Image dimensions
img_size = (256, 256)

# Batch size
batch_size = 32

In [15]:
num_classes = len(train_df['Class_Name'].unique())
print(num_classes)

832


In [61]:
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.utils import to_categorical

def train_data_generator(df, images_dir, batch_size):
    while True:
        batch_indices = np.random.choice(df.shape[0], size=batch_size, replace=False)
        batch_df = df.iloc[batch_indices]

        batch_images = []
        batch_labels = []

        for _, row in batch_df.iterrows():
            img_path = os.path.join(images_dir, row['Image_Name'])
            img = image.load_img(img_path, target_size=img_size)
            img_array = image.img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0)
            img_array = preprocess_input(img_array)

            batch_images.append(img_array)
            batch_labels.append(row['Class_Name'])

        # Convert batch labels to one-hot encoding
        batch_labels = to_categorical(batch_labels, num_classes=num_classes)



        yield np.vstack(batch_images), np.vstack(batch_labels)





In [42]:
# Create the ResNet50 model with pre-trained ImageNet weights
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
base_model.trainable = False

In [68]:
model = Sequential([
    base_model,
    Flatten(),
    Dense(500, activation='relu'),

    Dense(num_classes, activation='softmax')
])

In [44]:
print(y_label_encoded_validation.shape)

(4160, 832)


In [45]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 8, 8, 2048)        23587712  
                                                                 
 average_pooling2d_4 (Avera  (None, 4, 4, 2048)        0         
 gePooling2D)                                                    
                                                                 
 dense_11 (Dense)            (None, 4, 4, 1024)        2098176   
                                                                 
 dense_12 (Dense)            (None, 4, 4, 832)         852800    
                                                                 
Total params: 26538688 (101.24 MB)
Trainable params: 2950976 (11.26 MB)
Non-trainable params: 23587712 (89.98 MB)
_________________________________________________________________


In [69]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [70]:


# Create the training  data
train_generator = train_data_generator(train_df, train_images_dir, batch_size)



# Train the model
model.fit(train_generator, steps_per_epoch=len(train_df) // batch_size, epochs=30,validation_data=(x_validation,y_label_encoded_validation))



Epoch 1/30
1040/1040 [==============================] - 202s 170ms/step - loss: 6.8441 - accuracy: 0.0314 - val_loss: 6.6950 - val_accuracy: 0.0077
Epoch 2/30
  60/1040 [>.............................] - ETA: 2:25 - loss: 6.4219 - accuracy: 0.0453

KeyboardInterrupt: ignored